In [1]:
# Load symbolicregression model

import torch
import os, sys
import symbolicregression
import sympytorch
import requests
from sympy.core.rules import Transform

model_path = "ckpt/model.pt" 
try:
    if not os.path.isfile(model_path): 
        print("Downloading model...")
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        sr_model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        sr_model = torch.load(model_path)
        sr_model = sr_model.cuda()
    print(sr_model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    
    
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=sr_model,
                        max_input_points=10001,
                        n_trees_to_refine=5,
                        rescale=True
                        )

cuda:0
Model successfully loaded!


In [39]:
import numpy as np

X = 2 * np.random.randn(1000, 5)
y = np.exp(-2*(X[:, 0]-1)*(X[:, 0]-1)) #+ X[:, 0] ** 2 - 0.5

from pysr import PySRRegressor

model = PySRRegressor(
    niterations=100,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "log",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

model.fit(X, y)

print(model.get_best())
print(model.get_best()['equation'])

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/pysr/sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.680e+05
Head worker occupation: 5.5%
Progress: 367 / 1500 total iterations (24.467%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           9.760e-02  1.594e+01  y = 0.20244
2           9.760e-02  -0.000e+00  y = cos(1.3669)
4           9.734e-02  1.362e-03  y = (0.29007 * sin(x₀))
5           5.934e-02  4.948e-01  y = exp(-1.6813 + sin(x₀))
6           5.560e-02  6.519e-02  y = (exp(sin(inv(x₀))) * 0.20264)
7           4.244e-02  2.702e-01  y = exp(-1.6989 + sin(x₀ + 0.54666))
8           3.809e-02  1.081e-01  y = (exp(sin(x₀) + cos(x₀)) * 0.16868)
9           2.872e-02  2.822e-01  y = (0.27073 * (exp(sin(x₀ + 0.52768)) + -0.61103))
10          1.734e-02  5.047e-01  y = (0.40242 * (exp(sin(sin(x₀) + x₀)) + -0.68955))
11          1.468e-02  1.667e-01  y = (sin(sin(x₀) + x₀) * sin(exp(-0.86791 + x₀)))
12          1.389e-02  5.485e-02  y = 

complexity                                                      20
loss                                                      0.000358
score                                                     0.371766
equation         exp((((-1.9630105 + sin(x0 * 1.5123535)) * 1.0...
sympy_format     0.0352096521192219*exp(1.70474637096122*sin(1....
lambda_format    PySRFunction(X=>0.0352096521192219*exp(1.70474...
Name: 14, dtype: object
exp((((-1.9630105 + sin(x0 * 1.5123535)) * 1.0358834) + sin(cos(sin(x0)) + (x0 * 0.9229991))) * 1.6456933)


In [27]:
model.get_best()['sympy_format']

0.7139204*exp(1/(exp(x0) + 1/x0))*exp(sin(x0))*sin(x0)**2

In [3]:
# # from transformers import AutoTokenizer
# # CONTEXT_LENGTH = 256
# # tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") 
# # tokenizer.pad_token = tokenizer.eos_token
# # from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
# # model = AutoModelForCausalLM.from_pretrained("datasets/normalize_symbolic_regression_results_20231215/gptneo-350m-5500.model")


# from transformers import T5Tokenizer, DataCollatorForSeq2Seq
# from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# # Load the tokenizer, model, and data collator
# MODEL_NAME = "google/flan-t5-base"

# tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model = T5ForConditionalGeneration.from_pretrained("datasets/normalize_symbolic_regression_results_flant5_20231219/flant5-base-36000-loss0.097.model")
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [2]:
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot
from IPython.display import display

# Given an expr f (of variable t), returns its integral, together with t's and y's for regression
def integrate_expr(f, min_x=-4.0, max_x=4.0, increment=0.002, verbose=False):
    if verbose:
        print("Running integration on")
        display(f)
    # Compute integration
    x, t = symbols(['x','t'])
    fi = integrate(f, t)
    if verbose:
        display(fi)
        #plot(fi, (t, min_x, max_x))
    # Generate data for symbolic regression
    fl = lambdify((t), fi, "numpy")
    ts = np.arange(min_x, max_x, increment)
    ys = fl(ts)
    return fi, ts, ys
    
integrate_expr(sympify("2*t**2-t*2+2.5"), verbose=False)

(0.666666666666667*t**3 - 1.0*t**2 + 2.5*t,
 array([-4.   , -3.998, -3.996, ...,  3.994,  3.996,  3.998]),
 array([-68.66666667, -68.58170266, -68.49681062, ...,  36.50791852,
         36.56077862,  36.61369466]))

In [10]:
from utils.utils import *

def round_expr(expr, num_digits=2):
    return expr.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))

# Run symbolic regression on given data
# Returns: (raw regressed expr, rounded expr, model refined expr)
@timeout(15)
def symbolic_regress(sr_model, xs, ys, generate_refinement=False, verbose=False):
    if verbose:
        print("Running Symbolic Regression...")
    ##Example of data
    xs = np.reshape(xs, (len(xs),1))
    ys = np.reshape(ys, (len(ys),1))
    sr_model.fit(xs,ys)
    #
    replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
    model_str = sr_model.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
    for op,replace_op in replace_ops.items():
        model_str = model_str.replace(op,replace_op)
    #
    raw_expr = sp.parse_expr(model_str)
    x_0, t = symbols(['x_0', 't'])
    raw_expr = raw_expr.subs(x_0, t)
    if verbose:
        display(raw_expr)
    #
    expr = sp.expand(raw_expr)
    rounded_expr = round_expr(expr)
    if verbose:
        display(rounded_expr)
    #
    # Encode some input text
    if generate_refinement:
        prompt = str(rounded_expr)
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        #
        # Generate text
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.1)
        #
        # Decode and print the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_expr = sympify(generated_text)
        if verbose:
            display(generated_expr)
        #
    else:
        generated_expr = None
    return raw_expr, rounded_expr, generated_expr


fi, ts, ys = integrate_expr(sympify("sin(2*t)"), verbose=False)
rounded_fi = round_expr(fi)
display(rounded_fi)
raw_expr, rounded_expr, generated_expr = symbolic_regress(est, ts, ys, verbose=False)
display(sp.simplify(raw_expr))

print("Diff1:")
display(rounded_fi-rounded_expr)
# print("Diff2:")
# display(rounded_fi-generated_expr)

-cos(2*t)/2

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


0.00083 - 0.019687222780784245*((((0.050047730400721443*(t + 0.01084534112526919)**2 - 1)**2 + 0.010847738120465391)**2 + 0.0022916234231588482)**2 + 0.0082303798791981682)**3

Diff1:


-0.01*t**6 + 0.02*t**4 - 0.03*t**2 - cos(2*t)/2 + 0.02

In [5]:
sp.simplify(raw_expr)

0.66666570549541669*t**3 - 1.0000085584442815*t**2 + 2.4999751947224084*t - 2.3345401216920802e-5

In [4]:
import json
import random
import numpy as np

def load_expressions(filepaths):
    lines = []
    for filepath in filepaths:
        fin = open(filepath, 'r')
        lines.extend(fin.readlines())
        fin.close()
    #
    random.shuffle(lines)
    exprs = set()
    for line in lines:
        data = json.loads(line)
        for k,v in data.items():
            if k in ('f_t', 'g_t'):
                try:
                    if 'sqrt' not in v[1]:
                        expr = sympify(v[1])
                        exprs.add(expr)
                except:
                    continue
            elif k in ('original'):
                try:
                    if 'sqrt' not in v:
                        expr = sympify(v)
                        exprs.add(expr)
                except:
                    continue
    return exprs

exprs = load_expressions(['datasets/parametric_equations_pairs.json'])  #'datasets/parametric_equations.json'  'datasets/function_evaluation.json', 

In [5]:
len(exprs)

81647

In [ ]:
# # Run symbolic regression on each case

# seen_exprs = set()
# fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
# lines = fin.readlines()
# for line in lines:
#     result = json.loads(line)
#     expr = result["original"]
#     seen_exprs.add(expr)
# fin.close()
# print(f"{len(seen_exprs)} exprs loaded")

# fout = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "a")
# num_seen = 0
# num_seen_changed = False

# for f in exprs:
#     if str(f) in seen_exprs:
#         num_seen+=1
#         num_seen_changed = True
#         continue
#     else:
#         num_seen_changed = False
#     if num_seen_changed:
#         print(f"{num_seen} exprs ignored")
#     #print("Original expr and its integral:")
#     #display(f)
#     #print(f)
#     try:
#         fi, xs, ys = integrate_expr(f, verbose=False)
#         x, t = symbols(['x','t'])
#         fi = fi.subs({x:t})
#         rounded_fi = round_expr(fi)
#         #display(rounded_fi)
#         raw_expr, rounded_expr, generated_expr = symbolic_regress(est, xs, ys, generate_refinement=False, verbose=False)
# #         print("Generated expr:")
# #         display(generated_expr)
#         results = {"original":str(f),
#                    "integral":str(fi),
#                    "rounded_integral":str(rounded_fi),
#                    "regressed":str(raw_expr),
#                    "rounded_regressed":str(rounded_expr),
#                    #"generated_regressed":str(generated_expr),
#                    "diff_rounded": str(rounded_fi-rounded_expr),
#                    #"diff_generated": str(rounded_fi-generated_expr)
#                   }
#         fout.write(json.dumps(results))
#         fout.write('\n')
#         fout.flush()
#     except:
#         print("Failed to run symbolic regression")
#         continue
#     #     print("Diff1:")
#     #     display(rounded_fi-rounded_expr)
#     #     print("Diff2:")
#     #     display(rounded_fi-generated_expr)

# fout.close()

31867 exprs loaded


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

In [4]:
# Check the accuracy of symbolic regression

import json
from sympy import evalf, N
from utils.utils import *


# Check if f1 and f2 are almost equal.
# Note: Relative error is defined based on f1. Please use the original expression as f1.
def almost_equal(f1, f2, max_abs_error=0.011, max_relative_error=0.011, verbose=False):
    expr = f1-f2
    coeff_pairs = None
    try:
        coeff_pairs = get_coefficients_and_exponents(expr)
    except:
        print("Cannot get_coefficients_and_exponents")
        print(str(expr))
    if coeff_pairs is None:
        constants = get_all_constants(expr)
    else:
        constants = [p[0] for p in coeff_pairs]
    # Check if all diffs are within max_abs_error
    violators = [c for c in constants if c == sp.nan or abs(c) > max_abs_error]
    if verbose:
        print("Violating constants:", violators)
    if len(violators) == 0:
        return True
    # Check if all violating diffs are within max_relative_error
    try:
        coeffs1 = get_polynomial_coeffs(f1,max_power=7)
        coeffs2 = get_polynomial_coeffs(f2,max_power=7)
    except:
        return False
    for i in range(len(coeffs1)):
        if abs(coeffs1[i] - coeffs2[i]) > max_abs_error and \
           abs(coeffs1[i] - coeffs2[i]) > max_relative_error*abs(coeffs1[i]):
            return False
    return True
    

# fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
# lines = fin.readlines()

# num_total, qualified_rounded, qualified_generated = 0, 0, 0
# for line in lines:
#     result = json.loads(line)
#     if "diff_rounded" not in result or "diff_generated" not in result:
#         continue
#     rounded_integral = N(sympify(result["rounded_integral"]))
#     rounded_regressed = N(sympify(result["rounded_regressed"]))
# #     display(rounded_integral)
# #     display(rounded_regressed)
#     try:
#         rounded_regressed = filter_non_polynomial(rounded_regressed)
#     except:
#         print("Cannot filter non-polynomials on", str(rounded_regressed))
#     #generated_regressed = N(sympify(result["generated_regressed"]))
#     diff_rounded = rounded_integral - rounded_regressed
#     #diff_generated = sympify(result["diff_generated"])
#     num_total += 1
#     if almost_equal(rounded_integral, rounded_regressed, verbose=False):
#         qualified_rounded += 1
#     else:
#         display(rounded_integral)
#         display(rounded_regressed)
#         print(rounded_regressed)
#         display(diff_rounded)
# #     if is_close_to_zero(diff_generated, True):
# #         qualified_generated += 1
    
# fin.close()

# print(num_total, qualified_rounded, qualified_generated)

In [ ]:
# for i in range(1000):
#     f = originals[i]
#     x, t = symbols(['x','t'])
#     fi = integrate(f, t)
#     g = integrals[i]
#     #fs, gs, diff = get_diff(fi, g, t)
#     # print(np.mean(np.abs(fs)))
#     # print(np.mean(np.abs(gs)))
#     # print(np.mean(np.abs(diff)))
#     relative_diff = get_avg_diff(fi, g, t)
#     if relative_diff > 0.01:
#         print("CASE", i)
#         display(fi)
#         display(g)
#         print(relative_diff)

In [5]:
# Generate data for regression to infer the rules for integral
# The data is in the following format:
# Let us suppose there are N regression problems, and MAX_POWER=6, which is 
#   the highest power allowed in our polynomials.
#   data_series is a list of 7 items, each corresponding to a particular power
#   Each item is a tuple of 2 elements, with the first being x's and the second
#   being y's for the regression problem for the corresponding power.
#   The y's is a 1-D array of size N, which is the coefficient of that particular power for
#   each of N regression problems.
#   The each of the x's is an array of size 14, which contains 
#   [0, coeff-power-0, 1, coeff-power-1, 2, coeff-power-2, ...]
# If we want to find the connection between x's and y's, we should find that
#   y_1 = x_1 (coeff-power-0)
#   y_2 = x_3 (coeff-power-1)
#   y_3 = x_5 (coeff-power-2)
#   y_4 = x_7 (coeff-power-3)
#   y_5 = x_9 (coeff-power-4)

from sympy import evalf, N
from utils.utils import *

fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
lines = fin.readlines()
fin.close()
fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
lines.extend(fin.readlines())
fin.close()

MAX_POWER = 7
MAX_AVG_DIFF = 0.01

data_series = [([],[])] * (MAX_POWER+1)
data_series = []
originals = []
integrals = []
t = Symbol('t')

for i in range(MAX_POWER+1):
    data_series.append((list(),list()))

for line in lines:
    result = json.loads(line)
    if "rounded_regressed" not in result:
        continue
    original = N(sympify(result["original"]))
    integral = N(sympify(result["rounded_regressed"]))
    try:
        original = filter_non_polynomial(original)
        integral = filter_non_polynomial(integral)
        original_integral = integrate(original, t)
        avg_diff = get_avg_diff(original_integral, integral, t)
        if avg_diff > MAX_AVG_DIFF:
            #print("Skipping due to diff=", avg_diff)
            #display(original_integral)
            #display(integral)
            continue
    except:
        print("Cannot filter non-polynomials on", str(integral))
        continue
    try:
        coeffs_original = get_polynomial_coeffs(original,max_power=7)
        coeffs_integral = get_polynomial_coeffs(integral,max_power=7)
    except:
        print("Cannot get_coefficients_and_exponents")
        display(integral)
        continue
    if original.is_constant():
        #print("Skipping", line)
        continue
    xs = list()
    for i in range(MAX_POWER+1):
        xs.append(i)
        xs.append(coeffs_original[i])
    for i in range(MAX_POWER+1):
        data_series[i][0].append(xs.copy())
        data_series[i][1].append(coeffs_integral[i])
    if len(data_series[0][1]) % 100 == 0:
        print(len(data_series[0][1]), "processed")
#     if len(data_series[0][1]) == 3844:
#         display(original)
#         print(original.is_constant())
#         display(sympify(result["original"]))
#         display(integral)
    originals.append(original)
    integrals.append(integral)
    
fin.close()


100 processed
200 processed
300 processed
400 processed
500 processed
600 processed
700 processed
800 processed
900 processed
1000 processed
1100 processed
1200 processed
1300 processed
1400 processed
1500 processed
1600 processed
1700 processed
1800 processed
1900 processed
2000 processed
2100 processed
2200 processed
2300 processed
2400 processed
2500 processed
2600 processed
2700 processed
2800 processed
2900 processed
3000 processed
3100 processed
3200 processed
3300 processed
3400 processed
3500 processed
3600 processed
3700 processed
3800 processed
3900 processed
4000 processed
4100 processed
4200 processed
4300 processed
4400 processed
4500 processed
4600 processed
4700 processed
4800 processed
4900 processed
5000 processed
5100 processed
5200 processed
5300 processed
5400 processed
5500 processed
5600 processed
5700 processed
5800 processed
5900 processed
6000 processed
6100 processed
6200 processed
6300 processed
6400 processed
6500 processed
6600 processed
6700 processed
6800

50700 processed
50800 processed
50900 processed
51000 processed
51100 processed
51200 processed
51300 processed
51400 processed
51500 processed
51600 processed
51700 processed
51800 processed
51900 processed
52000 processed
52100 processed
52200 processed
52300 processed
52400 processed


In [6]:
import pickle

fout = open('/tmp/test_data_series_20240118.pkl', 'wb')
pickle.dump(data_series, fout)
fout.close()

# fin = open('/tmp/test_data_series.pkl', 'rb')
# data_series = pickle.load(fin)
# fin.close()

# Check if the results are as expected
n = len(data_series[1][1])
for y_idx in range(3, 4):
    xs = np.asarray(data_series[y_idx][0])
    x1s = xs[:,(y_idx-1)*2+1]
    ys = data_series[y_idx][1]
    for i in range(n):
        if abs(ys[i] - x1s[i]/y_idx) > 0.02 and abs(ys[i]/(x1s[i]/y_idx)-1) > 0.02:
            print(f"y_idx={y_idx}, i={i}, x={x1s[i]}, y={ys[i]}")
            display(originals[i])
            display(integrals[i])

y_idx=3, i=237, x=0.16, y=0.03


0.16*t**2 - 6.88*t + 73.96

0.03*t**3 - 3.46*t**2 + 74.2*t

y_idx=3, i=909, x=0.1111111111111111, y=0.0


0.111111111111111*t**2 - 4.0*t + 36.0

...

-5.0*t**5 + 6.0*t**4 + 3.33*t**2 - 2.1*t + 4.94

-0.83*t**6 + 1.19*t**5 - 0.05*t**4 + 0.92*t**3 - 1.46*t**2 + 4.48*t - 0.22

y_idx=3, i=52479, x=0.0, y=1.92


4.14*t**6 - 3.13*t**3 - 4.73*t + 1.63

0.59*t**7 - 0.04*t**6 - 0.14*t**5 - 0.71*t**4 + 1.92*t**3 + 3.15*t**2 + 8.23*t + 2.92

In [ ]:
y_idx=1

n = len(data_series[1][1])

est_simple = symbolicregression.model.SymbolicTransformerRegressor(
                        model=sr_model,
                        max_input_points=10001,
                        n_trees_to_refine=100,
                        rescale=True
                        )


xs = np.asarray(data_series[y_idx][0])
x1s = np.reshape(xs[:,y_idx], (n,1))
ys = np.reshape(data_series[y_idx][1], (n,1))
ys = ys + 2*x1s*x1s + 3.5

print(xs[0:10, y_idx])
print(ys[0:10])

est_simple.fit(x1s,ys)

replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est_simple.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
    
print(model_str)

raw_expr = sp.parse_expr(model_str)
sp.simplify(sp.expand(round_expr(raw_expr)))

In [ ]:
y_idx=2
n = len(data_series[1][1])

xs = np.asarray(data_series[y_idx][0])
x3s = np.reshape(xs[:,2*y_idx+1], (n,1))
ys = np.reshape(data_series[y_idx][1], (n,1))
ys = ys + 2*x3s*x3s + 3.5

# print(xs[0:10, 1])
# print(ys[0:10])

est_simple.fit(x3s,ys)

replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est_simple.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
    
print(model_str)

raw_expr = sp.parse_expr(model_str)
sp.simplify(sp.expand(round_expr(raw_expr)))

In [10]:
import scipy

y_idx=1

for x_idx in range(7):
    print(y_idx,"vs", x_idx)
    ys = np.asarray(data_series[y_idx][1])
    xs = np.asarray(data_series[y_idx][0])[:,2*x_idx+1]
    print(scipy.stats.linregress(xs, ys))

# diff = (ys-xs).flatten()
# for i in range(len(diff)):
#     if abs(diff[i]) > 0.02:
#         print(i, diff[i])

1 vs 0
LinregressResult(slope=1.0000544493616448, intercept=-0.007200875579005839, rvalue=0.999961514884511, pvalue=0.0, stderr=3.829651816159767e-05, intercept_stderr=0.003187600328923242)
1 vs 1
LinregressResult(slope=0.007511395829484341, intercept=31.566141146197644, rvalue=0.0055568864220011715, pvalue=0.20297364262515685, stderr=0.005899893017089653, intercept_stderr=0.3362899304073773)
1 vs 2
LinregressResult(slope=0.747423787547246, intercept=24.908136341597938, rvalue=0.17655782258913538, pvalue=0.0, stderr=0.018187158720411784, intercept_stderr=0.368543429993657)
1 vs 3
LinregressResult(slope=0.09919815803627527, intercept=31.39559398503262, rvalue=0.03642062641906203, pvalue=7.004931405918051e-17, stderr=0.011880362209675002, intercept_stderr=0.33664933179970846)
1 vs 4
LinregressResult(slope=0.76529721709672, intercept=29.769093109997787, rvalue=0.0951751759125878, pvalue=6.995420806968347e-106, stderr=0.034937530842137215, intercept_stderr=0.3446765218516759)
1 vs 5
Linreg

In [23]:
np.asarray(data_series[y_idx][0])[:,3]

array([ 8.5       ,  4.33333333,  0.25      , ..., 16.        ,
       18.75      , -5.61      ])

In [ ]:
originals[4]